In [20]:
import pandas as pd
import requests
import json
from datetime import datetime
import sqlalchemy
from sqlalchemy import create_engine

In [21]:
#API token for exchangerate.host
api_token = "8ed177bf655d362a495251f500f37a35"

list_url = f"https://api.exchangerate.host/list?access_key={api_token}"

In [22]:
#to save the currency list as json
response = requests.get(list_url)

currency_list = response.json()

with open("currency_list.json", "w") as f:
    json.dump(currency_list, f)

In [23]:
#to get the current date
# today = datetime.today().strftime("%Y-%m-%d")

In [24]:
def extract_curr(base_currency="IDR"):
  # url = f"https://api.exchangerate.host/timeframe?access_key={api_token}&source={base_currency}&start_date=2025-09-01&end_date={today}"
  url = f"https://api.exchangerate.host/timeframe?access_key={api_token}&source={base_currency}&start_date=2025-09-01&end_date=2025-10-19"
  response = requests.get(url)

  data = response.json()

  #save the value as JSON
  with open("historical_curr.json", "w") as f:
    json.dump(data, f)

  return data


In [ ]:
def transform_curr(data):
  historical_curr = data['quotes']
  records = []
  #opening the dictionary
  for date, rates in historical_curr.items():
    for currency, rate in rates.items():
      records.append({
          "Target_Currency" : currency,
          "Base_Currency" : "IDR",
          "Date" : date,
          "Exchange_Rate" : rate
      })
  df = pd.DataFrame(records)

  df["Rate_in_IDR"] = 1 / df["Exchange_Rate"]
  df["Target_Currency"] = df['Target_Currency'].str.replace('IDR', '')

  # Reload the currency DataFrame from the JSON file
  with open("currency_list.json","r") as file:
    curr = json.load(file)
  currency_df = pd.DataFrame(curr["currencies"].items(), columns=["Target_Currency", "Description"])

  #merging the currecny dataframe with the current df based on Target_Currency
  df = df.merge(currency_df, on="Target_Currency", how="left")
  df["Created_at"] = df['Created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

  #Format the data types for each column
  df = df.astype({
      "Date" : "datetime64[ns]",
      "Target_Currency" : "string",
      "Exchange_Rate" : "float64",
      "Rate_in_IDR" : "float64",
      "Base_Currency" : "string",
      "Description" : "string",
      "Created_at" : "datetime64[ns]"
      })

  #Reorder the columns order
  column_order = ['Date','Target_Currency','Description','Exchange_Rate','Rate_in_IDR','Base_Currency','Created_at']
  df = df[column_order]

  return df

In [26]:
idr_rate = extract_curr('IDR')

In [32]:
df = transform_curr(idr_rate)

df.head()

,Date,Target_Currency,Description,Exchange_Rate,Rate_in_IDR,Base_Currency,Created_at
0,2025-09-01,AED,United Arab Emirates Dirham,0.000223,4484.304933,IDR,2025-10-20 02:46:23
1,2025-09-01,AFN,Afghan Afghani,0.004162,240.269101,IDR,2025-10-20 02:46:23
2,2025-09-01,ALL,Albanian Lek,0.005083,196.734212,IDR,2025-10-20 02:46:23
3,2025-09-01,AMD,Armenian Dram,0.023276,42.962708,IDR,2025-10-20 02:46:23
4,2025-09-01,ANG,Netherlands Antillean Guilder,0.000109,9174.311927,IDR,2025-10-20 02:46:23


In [28]:
print(df.dtypes)

Date               datetime64[ns]
Target_Currency    string[python]
Description        string[python]
Exchange_Rate             float64
Rate_in_IDR               float64
Base_Currency      string[python]
Created_at         datetime64[ns]
dtype: object


In [ ]:
def load_to_mysql(df):
  #connect the MySQL database
  usr = 'root'
  pwd = 'root'
  host = '127.0.0.1'
  port = 3306
  dbName = 'currency'
  tableName = 'exchange_rate'
  try:
    engine = create_engine(f"mysql+mysqldb://{usr}:{pwd}@{host}:{port}/{dbName}",
                           echo=True,
                           future=True)
    #Send the dataframes to MySQL table
    df.to_sql(name=tableName, con=engine, if_exists="replace", index=False)

    print("Data berhasil dimuat ke tabel 'exchange_rate' di MySQL")
  except Exception as e:
    print("Gagal memuat data ke mysql:",e)

In [30]:
df = load_to_mysql(df)

2025-10-20 02:45:24,994 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-10-20 02:45:24,995 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 02:45:24,996 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-10-20 02:45:24,997 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 02:45:24,998 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-10-20 02:45:24,999 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 02:45:25,000 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-20 02:45:25,002 INFO sqlalchemy.engine.Engine DESCRIBE `currency`.`exchange_rate`
2025-10-20 02:45:25,003 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 02:45:25,009 INFO sqlalchemy.engine.Engine DESCRIBE `currency`.`exchange_rate`
2025-10-20 02:45:25,010 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 02:45:25,012 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `currency`
2025-10-20 02:45:25,013 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 02:45:25,015 INFO s

In [ ]:
#Filtering the dataframes by Target_Currency with the values is JPY
df[df['Target_Currency'] == 'JPY']

,Date,Target_Currency,Description,Exchange_Rate,Rate_in_IDR,Base_Currency,Created_at
73,2025-09-01,JPY,Japanese Yen,0.008949,111.744329,IDR,2025-10-20 02:46:23
244,2025-09-02,JPY,Japanese Yen,0.009030,110.741971,IDR,2025-10-20 02:46:23
415,2025-09-03,JPY,Japanese Yen,0.008985,111.296605,IDR,2025-10-20 02:46:23
586,2025-09-04,JPY,Japanese Yen,0.009015,110.926234,IDR,2025-10-20 02:46:23
757,2025-09-05,JPY,Japanese Yen,0.008999,111.123458,IDR,2025-10-20 02:46:23
928,2025-09-06,JPY,Japanese Yen,0.009000,111.111111,IDR,2025-10-20 02:46:23
1099,2025-09-07,JPY,Japanese Yen,0.009046,110.546098,IDR,2025-10-20 02:46:23
1270,2025-09-08,JPY,Japanese Yen,0.008944,111.806798,IDR,2025-10-20 02:46:23
1441,2025-09-09,JPY,Japanese Yen,0.008957,111.644524,IDR,2025-10-20 02:46:23
1612,2025-09-10,JPY,Japanese Yen,0.008958,111.632061,IDR,2025-10-20 02:46:23
